### Data Collection


1. GET S&P 500 company info<br>


In [117]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [118]:
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from pathlib import Path
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jakek\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### 1. Get S&P 500 Data

In [119]:
# Get table of the S&P 500 tickers, CIK, and industry from Wikipedia
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cik_df = pd.read_html(wiki_url,header=0,index_col=0)[0]
cik_df['GICS Sector'] = cik_df['GICS Sector'].astype("category")
cik_df['GICS Sub Industry'] = cik_df['GICS Sector'].astype("category")
cik_df.tail(100)

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,GICS Sub Industry
Symbol,,,,,,,,,
RHI,Robert Half,reports,Industrials,Human Resource & Employment Services,"Menlo Park, California",2000-12-05,315213,1948,Industrials
ROK,Rockwell Automation,reports,Industrials,Electrical Components & Equipment,"Milwaukee, Wisconsin",NaN,1024478,1903,Industrials
ROL,Rollins,reports,Industrials,Environmental & Facilities Services,"Atlanta, Georgia",2018-10-01,84839,1948,Industrials
ROP,Roper,reports,Industrials,Industrial Conglomerates,"Sarasota, Florida",2009-12-23,882835,1981,Industrials
ROST,Ross,reports,Consumer Discretionary,Apparel Retail,"Dublin, California",2009-12-21,745732,1982,Consumer Discretionary
...,...,...,...,...,...,...,...,...,...
YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,Consumer Discretionary
ZBRA,Zebra,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,Information Technology
ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,Health Care


In [120]:
symbol_list = cik_df.index.tolist()
symbol_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [121]:
string_1 = ' '.join([symbol_list[i] for i in range(0,50)]).replace(" ", " OR ")
string_2 = ' '.join([symbol_list[i] for i in range(50,100)]).replace(" ", " OR ")
string_3 = ' '.join([symbol_list[i] for i in range(100,150)]).replace(" ", " OR ")
string_4 = ' '.join([symbol_list[i] for i in range(150,200)]).replace(" ", " OR ")
string_5 = ' '.join([symbol_list[i] for i in range(200,250)]).replace(" ", " OR ")
string_6 = ' '.join([symbol_list[i] for i in range(250,300)]).replace(" ", " OR ")
string_7 = ' '.join([symbol_list[i] for i in range(300,350)]).replace(" ", " OR ")
string_8 = ' '.join([symbol_list[i] for i in range(350,400)]).replace(" ", " OR ")
string_9 = ' '.join([symbol_list[i] for i in range(400,450)]).replace(" ", " OR ")
string_10 = ' '.join([symbol_list[i] for i in range(450,500)]).replace(" ", " OR ")

In [122]:
from newsapi import NewsApiClient
load_dotenv()
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

headlines = newsapi.get_everything(
    q=,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [123]:
headlines_1 = newsapi.get_everything(
    q = string_1,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_2 = newsapi.get_everything(
    q = string_2,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_3 = newsapi.get_everything(
    q = string_3,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_4 = newsapi.get_everything(
    q = string_4,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_5 = newsapi.get_everything(
    q = string_5,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_6 = newsapi.get_everything(
    q = string_6,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_7 = newsapi.get_everything(
    q = string_7,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_8 = newsapi.get_everything(
    q = string_8,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_9 = newsapi.get_everything(
    q = string_9,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_10 = newsapi.get_everything(
    q = string_10,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [124]:
# Create the S&p 500 sentiment scores DataFrame
sentiments_1 = []

for article in headlines_1["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        
        sentiments_1.append({
            "Text": text
        })
        
    except AttributeError:
        pass

sentiments_2 = []

for article in headlines_2["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]

        sentiments_2.append({
            "Text": text
        })
        
    except AttributeError:
        pass


sentiments_3 = []

for article in headlines_3["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        
        sentiments_3.append({
            "Text": text
        })
        
    except AttributeError:
        pass

sentiments_4 = []

for article in headlines_4["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
               
        sentiments_4.append({
            "Text": text
        })
        
    except AttributeError:
        pass


sentiments_5 = []
for article in headlines_5["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
       
        sentiments_5.append({
            "Text": text
        })
        
    except AttributeError:
        pass

sentiments_6 = []
for article in headlines_6["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        
        sentiments_6.append({
            "Text": text
        })
        
    except AttributeError:
        pass

sentiments_7 = []
for article in headlines_7["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)

        sentiments_7.append({
            "Text": text
        })
        
    except AttributeError:
        pass

sentiments_8 = []
for article in headlines_8["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]

        sentiments_8.append({
            "Text": text    
        })
        
    except AttributeError:
        pass

sentiments_9 = []
for article in headlines_9["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
      
        sentiments_9.append({
            "Text": text
            
        })
        
    except AttributeError:
        pass

sentiments_10 = []
for article in headlines_10["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        
        sentiments_10.append({
            "Text": text
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
df_1 = pd.DataFrame(sentiments_1)
df_2 = pd.DataFrame(sentiments_2)
df_3 = pd.DataFrame(sentiments_3)
df_4 = pd.DataFrame(sentiments_4)
df_5 = pd.DataFrame(sentiments_5)
df_6 = pd.DataFrame(sentiments_6)
df_7 = pd.DataFrame(sentiments_7)
df_8 = pd.DataFrame(sentiments_8)
df_9 = pd.DataFrame(sentiments_9)
df_10 = pd.DataFrame(sentiments_10)


# Reorder DataFrame columns
cols = ["Text"]
df_1 = df_1[cols]
df_2 = df_2[cols]
df_3 = df_3[cols]
df_4 = df_4[cols]
df_5 = df_5[cols]
df_6 = df_6[cols]
df_7 = df_7[cols]
df_8 = df_8[cols]
df_9 = df_9[cols]
df_10 = df_10[cols]

,Text
0,Market-leading tools maker Stanley Black &amp;...
1,Wall Street is reeling under extreme volatilit...
2,"On the snowy roads near Kursk, tanks and milit..."
3,Illustration by Alex Castro / The Verge\r\n\n ...
4,Thermo Fisher Scientific Inc.TMO has been gain...
...,...
95,"JERUSALEM, Feb 23 (Reuters) - Israel's stock e..."
96,With less than a week before Elden RingsFebrua...
97,Romelu Lukaku started on the bench against Lil...
98,Now that The Book of Boba Fett/The Mandalorian...


In [125]:
# Instantiate the lemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Expand the default stopwords list if necessary
sw = set(stopwords.words('english'))

In [126]:
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem


tokenizer(df_1.iloc[0]['Text'])
tokenizer(df_2.iloc[0]['Text'])
tokenizer(df_3.iloc[0]['Text'])
tokenizer(df_4.iloc[0]['Text'])
tokenizer(df_5.iloc[0]['Text'])
tokenizer(df_6.iloc[0]['Text'])
tokenizer(df_7.iloc[0]['Text'])
tokenizer(df_8.iloc[0]['Text'])
tokenizer(df_9.iloc[0]['Text'])
tokenizer(df_10.iloc[0]['Text'])

['toronto',
 'after',
 'month',
 'in',
 'limbo',
 'with',
 'toronto',
 'fc',
 'star',
 'striker',
 'jozy',
 'altidores',
 'move',
 'to',
 'the',
 'new',
 'england',
 'revolution',
 'wa',
 'finally',
 'confirmed',
 'mondaythe',
 'news',
 'came',
 'in',
 'rapidfire',
 'fashionone',
 'minute',
 'after',
 'tf',
 'char']

In [127]:
df_1['tokens'] = df_1['Text'].apply(tokenizer)
df_2['tokens'] = df_2['Text'].apply(tokenizer)
df_3['tokens'] = df_3['Text'].apply(tokenizer)
df_4['tokens'] = df_4['Text'].apply(tokenizer)
df_5['tokens'] = df_5['Text'].apply(tokenizer)
df_6['tokens'] = df_6['Text'].apply(tokenizer)
df_7['tokens'] = df_7['Text'].apply(tokenizer)
df_8['tokens'] = df_8['Text'].apply(tokenizer)
df_9['tokens'] = df_9['Text'].apply(tokenizer)
df_10['tokens'] = df_10['Text'].apply(tokenizer)
df_9

,Text,tokens
0,Market-leading tools maker Stanley Black &amp;...,"[marketleading, tool, maker, stanley, black, a..."
1,Wall Street is reeling under extreme volatilit...,"[wall, street, is, reeling, under, extreme, vo..."
2,"On the snowy roads near Kursk, tanks and milit...","[on, the, snowy, road, near, kursk, tank, and,..."
3,Illustration by Alex Castro / The Verge\r\n\n ...,"[illustration, by, alex, castro, the, verge, f..."
4,Thermo Fisher Scientific Inc.TMO has been gain...,"[thermo, fisher, scientific, inctmo, ha, been,..."
...,...,...
95,"JERUSALEM, Feb 23 (Reuters) - Israel's stock e...","[jerusalem, feb, reuters, israel, stock, excha..."
96,With less than a week before Elden RingsFebrua...,"[with, le, than, a, week, before, elden, rings..."
97,Romelu Lukaku started on the bench against Lil...,"[romelu, lukaku, started, on, the, bench, agai..."
98,Now that The Book of Boba Fett/The Mandalorian...,"[now, that, the, book, of, boba, fettthe, mand..."


In [128]:
import spacy
from spacy import displacy

In [129]:
nlp = spacy.load('en_core_web_sm')

In [130]:
text_1 = ' '.join(df_1['Text'])
text_2 = ' '.join(df_2['Text'])
text_3 = ' '.join(df_3['Text'])
text_4 = ' '.join(df_4['Text'])
text_5 = ' '.join(df_5['Text'])
text_6 = ' '.join(df_6['Text'])
text_7 = ' '.join(df_7['Text'])
text_8 = ' '.join(df_8['Text'])
text_9 = ' '.join(df_9['Text'])
text_10 = ' '.join(df_10['Text'])
text_9

'Market-leading tools maker Stanley Black &amp; Decker (SWK) has been paying dividends to shareholders for the past 54 consecutive years while maintaining a dividend-growth streak. But is it wise to b… [+4468 chars] Wall Street is reeling under extreme volatility since the beginning of 2022. Investors are highly concerned about soaring inflation. Moreover, the uncertainty regarding the pace and magnitude of an i… [+7582 chars] On the snowy roads near Kursk, tanks and military equipment stop traffic. Videos from around the Russian cityroughly 100 miles from the border with Ukraineshow cars waiting in line to cross train tra… [+2929 chars] Illustration by Alex Castro / The Verge\r\n\n \n\n For the first time, Uber is allowing its customers to see how many five-star ratings they’ve received from drivers — as well as how many shameful one-sta… [+3875 chars] Thermo Fisher Scientific Inc.TMO has been gaining from strength across the majority of its end markets. The company ended the fourth q

In [131]:
doc_1 = nlp(text_1)
doc_2 = nlp(text_2)
doc_3 = nlp(text_3)
doc_4 = nlp(text_4)
doc_5 = nlp(text_5)
doc_6 = nlp(text_6)
doc_7 = nlp(text_7)
doc_8 = nlp(text_8)
doc_9 = nlp(text_9)
doc_10 = nlp(text_10)

In [132]:
# Add a title to the document
doc_1.user_data["title"] = "NER"

In [133]:
# Render the visualization
displacy.render(doc_1, style='ent')
displacy.render(doc_2, style='ent')
displacy.render(doc_3, style='ent')
displacy.render(doc_4, style='ent')
displacy.render(doc_5, style='ent')
displacy.render(doc_6, style='ent')
displacy.render(doc_7, style='ent')
displacy.render(doc_8, style='ent')
displacy.render(doc_9, style='ent')
displacy.render(doc_10, style='ent')

In [134]:
from collections import Counter

# List all Entities
org_list_1 = []

for ent in doc_1.ents:
    if ent.label_ == 'ORG':
        org_list_1.append(ent.text)

org_list_2 = []

for ent in doc_2.ents:
    if ent.label_ == 'ORG':
        org_list_2.append(ent.text)

org_list_3 = []

for ent in doc_3.ents:
    if ent.label_ == 'ORG':
        org_list_3.append(ent.text)

org_list_4 = []

for ent in doc_4.ents:
    if ent.label_ == 'ORG':
        org_list_4.append(ent.text)

org_list_5 = []

for ent in doc_5.ents:
    if ent.label_ == 'ORG':
        org_list_5.append(ent.text)

org_list_6 = []

for ent in doc_6.ents:
    if ent.label_ == 'ORG':
        org_list_6.append(ent.text)

org_list_7 = []

for ent in doc_1.ents:
    if ent.label_ == 'ORG':
        org_list_7.append(ent.text)

org_list_8 = []

for ent in doc_8.ents:
    if ent.label_ == 'ORG':
        org_list_8.append(ent.text)

org_list_9 = []

for ent in doc_9.ents:
    if ent.label_ == 'ORG':
        org_list_9.append(ent.text)

org_list_10 = []

for ent in doc_10.ents:
    if ent.label_ == 'ORG':
        org_list_10.append(ent.text)



org_list_1 = Counter(org_list_1).most_common(15)
org_list_2 = Counter(org_list_2).most_common(15)
org_list_3 = Counter(org_list_3).most_common(15)
org_list_4 = Counter(org_list_4).most_common(15)
org_list_5 = Counter(org_list_5).most_common(15)
org_list_6 = Counter(org_list_6).most_common(15)
org_list_7 = Counter(org_list_7).most_common(15)
org_list_8 = Counter(org_list_8).most_common(15)
org_list_9 = Counter(org_list_9).most_common(15)
org_list_10 = Counter(org_list_10).most_common(15)

df_1_org = pd.DataFrame(org_list_1, columns = ['text', 'count'])
df_2_org = pd.DataFrame(org_list_2, columns = ['text', 'count'])
df_3_org = pd.DataFrame(org_list_3, columns = ['text', 'count'])
df_4_org = pd.DataFrame(org_list_4, columns = ['text', 'count'])
df_5_org = pd.DataFrame(org_list_5, columns = ['text', 'count'])
df_6_org = pd.DataFrame(org_list_6, columns = ['text', 'count'])
df_7_org = pd.DataFrame(org_list_7, columns = ['text', 'count'])
df_8_org = pd.DataFrame(org_list_8, columns = ['text', 'count'])
df_9_org = pd.DataFrame(org_list_9, columns = ['text', 'count'])
df_10_org = pd.DataFrame(org_list_10, columns = ['text', 'count'])



frames = [df_1_org, df_2_org, df_3_org, df_4_org, df_5_org, df_6_org, df_7_org, df_8_org, df_9_org, df_10_org]
org_total = pd.concat(frames)

In [135]:
org_total = org_total.reset_index(drop=True).sort_values('count', ascending = False).groupby(['text', 'count'], as_index=False)['count'].sum()
org_total = org_total.groupby(['text']).sum().sort_values('count',ascending=False).reset_index()
org_total['checking'] = org_total.text.isin(cik_df['Security']).astype(int)
org_total = org_total[org_total['checking'] !=0]
org_total

,text,count,checking
4,Dow,14,1
6,Apple,11,1
10,Amazon,6,1
16,Tesla,4,1
23,AMD,4,1
43,Ventas,2,1
71,Meta,2,1
83,Copart,1,1


In [136]:
dow_headlines = newsapi.get_everything(
    q = "Dow",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

apple_headlines = newsapi.get_everything(
    q = "Apple",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

amazon_headlines = newsapi.get_everything(
    q = "Amazon",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

tesla_headlines = newsapi.get_everything(
    q = "Tesla",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

amd_headlines = newsapi.get_everything(
    q = "AMD",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [137]:
dow_news_df = pd.DataFrame.from_dict(dow_headlines["articles"])
apple_news_df = pd.DataFrame.from_dict(apple_headlines["articles"])
amazon_news_df = pd.DataFrame.from_dict(amazon_headlines["articles"])
tesla_news_df = pd.DataFrame.from_dict(tesla_headlines["articles"])
amd_news_df = pd.DataFrame.from_dict(amd_headlines["articles"])

In [138]:
# Create the Bitcoin sentiment scores DataFrame
dow_sentiments = []

for article in dow_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        dow_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

apple_sentiments = []

for article in apple_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        apple_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

amazon_sentiments = []

for article in amazon_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        amazon_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

tesla_sentiments = []

for article in tesla_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tesla_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

amd_sentiments = []

for article in amd_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        amd_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass


# Create DataFrame
dow_df = pd.DataFrame(dow_sentiments)
apple_df = pd.DataFrame(apple_sentiments)
amazon_df = pd.DataFrame(amazon_sentiments)
tesla_df = pd.DataFrame(tesla_sentiments)
amd_df = pd.DataFrame(amd_sentiments)


# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
dow_df = dow_df[cols]
apple_df = apple_df[cols]
amazon_df = amazon_df[cols]
tesla_df = tesla_df[cols]
amd_df = amd_df[cols]

In [139]:
dow_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,-0.107160,0.061870,0.084630,0.853500
std,0.465307,0.062606,0.078463,0.088323
min,-0.949300,0.000000,0.000000,0.628000
25%,-0.453775,0.000000,0.000000,0.787000
50%,0.000000,0.057500,0.073000,0.852000
75%,0.184000,0.106500,0.137000,0.921500
max,0.853100,0.250000,0.343000,1.000000


In [140]:
apple_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.200903,0.068770,0.023570,0.907690
std,0.362215,0.074522,0.046279,0.085798
min,-0.822500,0.000000,0.000000,0.664000
25%,0.000000,0.000000,0.000000,0.853750
50%,0.102700,0.065000,0.000000,0.924000
75%,0.506900,0.125500,0.045250,1.000000
max,0.827100,0.272000,0.255000,1.000000


In [141]:
amazon_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.267730,0.104410,0.041940,0.853690
std,0.396728,0.081048,0.084037,0.106428
min,-0.802000,0.000000,0.000000,0.312000
25%,0.000000,0.046750,0.000000,0.798250
50%,0.177900,0.095500,0.000000,0.861000
75%,0.636900,0.150000,0.063500,0.934250
max,0.966600,0.403000,0.688000,1.000000


In [142]:
tesla_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.073584,0.073520,0.056000,0.870450
std,0.482014,0.067883,0.077034,0.086036
min,-0.904200,0.000000,0.000000,0.683000
25%,-0.278900,0.000000,0.000000,0.833000
50%,0.012900,0.063000,0.000000,0.876000
75%,0.476700,0.114000,0.078750,0.934250
max,0.862200,0.261000,0.283000,1.000000


In [143]:
amd_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.201917,0.076850,0.026230,0.896930
std,0.387863,0.080606,0.040331,0.084551
min,-0.611400,0.000000,0.000000,0.678000
25%,0.000000,0.000000,0.000000,0.854250
50%,0.128000,0.067000,0.000000,0.893000
75%,0.549700,0.125750,0.056500,1.000000
max,0.923100,0.322000,0.159000,1.000000
